# Important Note:

### I strongly suggest you follow my step-by-step video tutorial. The video will not only walk you through each step of the process but also includes a real-time execution using Diablo 2 as an example.

### Here is the link to the video: 
https://www.youtube.com/watch?v=RSXgyDf2ALo
[![Watch the video](https://img.youtube.com/vi/RSXgyDf2ALo/maxresdefault.jpg)](https://youtu.be/RSXgyDf2ALo)


# **Instructions**:

### 1 - Open your game that you want to perform detections
### 2 - In the game window, get the name of it's title bar 
### 3 - Update the variable "window_name" with the game title bar name
### 4 - Run all cells to start generating your dataset in the folder images

In [5]:
import numpy as np
import win32gui, win32ui, win32con
from PIL import Image
from time import sleep
import os

In [3]:
import win32gui

def enum_windows_callback(hwnd, window_list):
    if win32gui.IsWindowVisible(hwnd):  # 只列出可见窗口
        title = win32gui.GetWindowText(hwnd)
        if title:  # 排除空标题窗口
            window_list.append((hwnd, title))

def list_windows():
    windows = []
    win32gui.EnumWindows(enum_windows_callback, windows)
    for hwnd, title in windows:
        print(f"窗口句柄: {hwnd}, 标题: {title}")

list_windows()

窗口句柄: 266102, 标题: 1_generate_dataset.ipynb - Untitled (Workspace) - Visual Studio Code
窗口句柄: 135286, 标题: MAME: 三国战纪 - 风云再起 (版本 101, CN) [kovsh101]
窗口句柄: 920368, 标题: Microsoft Edge 的新增功能 - 个人 - Microsoft​ Edge
窗口句柄: 67742, 标题: OpenVPN Connect
窗口句柄: 131334, 标题: mame模拟器怎么暂停游戏_百度搜索 - Google Chrome
窗口句柄: 134540, 标题: ~/project/marketsystemui/yarn.lock - marketsystemui (Workspace) [WSL: ubuntu-22.04] - Cursor
窗口句柄: 789116, 标题: 三国战纪1、2合集（整合Mame32 模拟器） - 文件资源管理器
窗口句柄: 134550, 标题: ~/project/social-marketing/src/background.ts - social-marketing (Workspace) [WSL: ubuntu-22.04] - Cursor
窗口句柄: 134536, 标题: ~/project/langgraphdemo/MultiAgent/Director.py - langgraphdemo (Workspace) [WSL: ubuntu-22.04] - Cursor
窗口句柄: 200412, 标题: ~/project/go_workspace/gindemo/channeltest.go - Untitled (Workspace) - Cursor
窗口句柄: 396514, 标题: ~/project/yolo/yolov3/data/voc.yaml - yolov3 (Workspace) [WSL: ubuntu-22.04] - Cursor
窗口句柄: 330660, 标题: ~/project/aiFetchly/doc/mcp-server-prd.md - aiFetchly (Workspace) [WSL: ubuntu-

In [5]:
class WindowCapture:
    w = 0
    h = 0
    hwnd = None

    def __init__(self, window_name):
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))
        else:
            print('Window found: {}'.format(window_name))
        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

    def get_screenshot(self):
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.fromstring(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[...,:3]
        img = np.ascontiguousarray(img) 
            
        return img

    def generate_image_dataset(self):
        if not os.path.exists("images"):
            os.mkdir("images")
        while(True):
            img = self.get_screenshot()
            im = Image.fromarray(img[..., [2, 1, 0]])
            im.save(f"./images/img_{len(os.listdir('images'))}.jpg")
            sleep(0.3)
    
    def get_window_size(self):
        return (self.w, self.h)

In [6]:
# Execute this cell to generate a dataset of images for the specified window.
window_name = "MAME: 三国战纪 - 风云再起 (版本 101, CN) [kovsh101]"

wincap = WindowCapture(window_name)
wincap.generate_image_dataset()

Window found: MAME: 三国战纪 - 风云再起 (版本 101, CN) [kovsh101]


C:\Users\zengj\AppData\Local\Temp\ipykernel_3248\1096205031.py:33: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(signedIntsArray, dtype='uint8')


error: (1400, 'GetWindowDC', '无效的窗口句柄。')

In [1]:
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    dot_product = np.dot(a, b.T)
    norm_a = np.linalg.norm(a, axis=1, keepdims=True)
    norm_b = np.linalg.norm(b, axis=1, keepdims=True)
    similarity = dot_product / (norm_a * norm_b.T)
    return similarity


In [ ]:
from shutil import copy2

def find_and_copy_unique_images(src_folder='./images', dst_folder='unique_image', threshold=0.5):
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    image_files = [f for f in os.listdir(src_folder) if f.lower().endswith('.jpg')]
    features = []
    unique_files = []

    for img_file in image_files:
        img_path = os.path.join(src_folder, img_file)
        img = Image.open(img_path).resize((64, 64)).convert('RGB')
        feature = np.array(img).flatten()
        is_unique = True
        if features:
            sims = cosine_similarity([feature], np.array(features))
            if np.max(sims) > threshold:
                is_unique = False
        if is_unique:
            features.append(feature)
            unique_files.append(img_file)
            copy2(img_path, os.path.join(dst_folder, img_file))
    print(f"Copied {len(unique_files)} unique images to '{dst_folder}'.")

# Usage:
find_and_copy_unique_images()

C:\Users\zengj\AppData\Local\Temp\ipykernel_39044\3039999235.py:7: RuntimeWarning: invalid value encountered in divide
  similarity = dot_product / (norm_a * norm_b.T)


Copied 532 unique images to 'unique_image'.


: 